In [123]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import pandas as pd
import ipywidgets as widgets
from ipywidgets import *

In [92]:
loc = '/Users/arjun17/Downloads/insurance.csv'

In [93]:
df = pd.read_csv(loc)

In [134]:
df_orig = pd.read_csv(loc)

In [94]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [95]:
encoder = LabelEncoder()

In [96]:
df['sex'] = encoder.fit_transform(df['sex'])
df['smoker'] = encoder.fit_transform(df['smoker'])
df['region'] = encoder.fit_transform(df['region'])

In [99]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,3,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.000,3,0,2,4449.46200
3,33,1,22.705,0,0,1,21984.47061
4,32,1,28.880,0,0,1,3866.85520


In [100]:
@interact
def correlations(column1 = df.select_dtypes('number').columns, column2 = df.select_dtypes('number').columns):
    print(f"Correlation: {df[column1].corr(df[column2])}")

interactive(children=(Dropdown(description='column1', options=('age', 'sex', 'bmi', 'children', 'smoker', 'reg…

In [101]:
X = df[['age', 'bmi', 'smoker']]

In [102]:
X.head()

,age,bmi,smoker
0,19,27.900,1
1,18,33.770,0
2,28,33.000,0
3,33,22.705,0
4,32,28.880,0


In [103]:
target = df['charges']

In [104]:
target.head()

0    16884.92400
1     1725.55230
2     4449.46200
3    21984.47061
4     3866.85520
Name: charges, dtype: float64

In [115]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, target, test_size = 0.2)

In [116]:
model = LinearRegression()

In [117]:
model.fit(Xtrain,Ytrain)

LinearRegression()

In [122]:
y_pred_training = model.predict(Xtest)
print("Training MAE:", mean_absolute_error(y_pred_training, Ytest))

Training MAE: 3791.9962186922594


In [129]:
def make_prediction(age, bmi, smoker):
    df = pd.DataFrame({"age":age, "bmi":bmi, "smoker":smoker},index=[0])
    prediction = model.predict(df).round(2)[0]
    return f"Predicted total insurance cost: ${prediction}"

In [130]:
make_prediction(39, 20, 0)

'Predicted total insurance cost: $4948.23'

In [138]:
print("Smoker: \nYes = 1 \nNo = 0")
widgets.interact(
                make_prediction,
    
                age=IntSlider(
                    min=X["age"].min(),
                    max=X["age"].max(),
                    value=X["age"].mean(),
                ),
                bmi= widgets.FloatSlider(
                    min=X["bmi"].min(),
                    max=X["bmi"].max(),
                    step=0.01,
                    value=X["bmi"].mean(),
                ),
                smoker= widgets.Dropdown(options=sorted(X["smoker"].unique())),
            );

Smoker: 
Yes = 1 
No = 0


interactive(children=(IntSlider(value=39, description='age', max=64, min=18), FloatSlider(value=30.66339686098…